Train a neural network in the simplest way possible.  
Not using dataloader.  
Using CIFAR10 data but starting from this data as a numpy array!!!

check this cool titorial: https://stanford.edu/~shervine/blog/pytorch-how-to-generate-data-parallel

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np

In [2]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [3]:
import torch.nn as nn
import torch.nn.functional as F
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
net = Net()

In [4]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [5]:
print(type(trainset.data))
print(trainset.data.shape)
print("")
print(type(trainset.targets))
print(len(trainset.targets))

<class 'numpy.ndarray'>
(50000, 32, 32, 3)

<class 'list'>
50000


In [6]:
# get data and targets
trainData = trainset.data
trainTarg = np.asarray(trainset.targets)

# shuffle them
indexes = np.random.permutation(trainTarg.shape[0])
trainData = trainData[indexes, :, :, :]
trainTarg = trainTarg[indexes]

In [7]:
# move axis of the trainData array
trainData = np.moveaxis(trainData, 3, 1)
print(trainData.shape)

(50000, 3, 32, 32)


In [8]:
# # check that moveaxis works 
# w = trainData[0, :, :, 2]
# e = q[0, 2, :, :]
# np.array_equal(w, e)

In [9]:
# normalize train data
trainData = trainData/255 * 2 - 1 

In [10]:
# train the model
for epoch in range(2):
    running_loss = 0.0
    for i in range(12500):
        # Local batches and labels
        inputs, labels = trainData[i*4:(i+1)*4, ], trainTarg[i*4:(i+1)*4, ]
        inputs = torch.from_numpy(inputs)
        labels = torch.from_numpy(labels)
        
#         print(type(inputs))
#         print(inputs.shape)
#         print("")
#         print(type(labels))
#         print(labels.shape)
        
#         break

        # zero the parameter gradients
        optimizer.zero_grad()
        
        # forward + backward + optimize
        outputs = net(inputs.float())
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0


[1,  2000] loss: 2.167
[1,  4000] loss: 1.793
[1,  6000] loss: 1.637
[1,  8000] loss: 1.568
[1, 10000] loss: 1.492
[1, 12000] loss: 1.453
[2,  2000] loss: 1.383
[2,  4000] loss: 1.366
[2,  6000] loss: 1.331
[2,  8000] loss: 1.315
[2, 10000] loss: 1.276
[2, 12000] loss: 1.260
